In [3]:
%%configure -f
{"conf": {"spark.jars": "/jar/simpleHTM.jar,/jar/jhealpix.jar"}}

In [4]:
import simple.HTMindex
import healpix.jhu.Healpix
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf

//var htmindex= new HTMindex()
//val htmid: (Double, Double) => Long = htmindex.lookupId(_,_)

//val htmidUDF=udf(htmid)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1576183992464_0008,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import simple.HTMindex
import healpix.jhu.Healpix
import org.apache.spark.sql.{SparkSession, SaveMode, Row, DataFrame}
import org.apache.spark.sql.functions.udf


In [5]:
object HTMUtils extends Serializable {
    var htmindex= new HTMindex() with Serializable
    val htmid: (Double, Double) => Long = htmindex.lookupId(_,_)

    val htmidUDF=udf(htmid)
    
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object HTMUtils


In [6]:
object HEALPixUtils extends Serializable {
    var hp = new Healpix() with Serializable
    val healpixid: (Double, Double) => Long = hp.ang2pix(_,_)

    val healpixidUDF=udf(healpixid)
    
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object HEALPixUtils


In [7]:
val dataset = Seq((123.0,45.0), (124.0,46.0)).toDF("ra", "dec")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dataset: org.apache.spark.sql.DataFrame = [ra: double, dec: double]


In [8]:
dataset.withColumn("htmid", htmidUDF(dataset("ra"),dataset("dec"))).show

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<console>:30: error: not found: value htmidUDF
       dataset.withColumn("htmid", htmidUDF(dataset("ra"),dataset("dec"))).show
                                   ^



In [9]:
val datafile = "/user/hive/warehouse/source_test_parquet"
val old_df = spark.read.parquet(datafile)

old_df.createOrReplaceTempView("source")

val coordsDF = spark.sql("select coord_ra, coord_decl from source limit 10")
coordsDF.show()
coordsDF.printSchema()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datafile: String = /user/hive/warehouse/source_test_parquet
old_df: org.apache.spark.sql.DataFrame = [id: bigint, chunkid: int ... 86 more fields]
coordsDF: org.apache.spark.sql.DataFrame = [coord_ra: double, coord_decl: double]
+------------------+------------------+
|          coord_ra|        coord_decl|
+------------------+------------------+
| 339.8799009480773| 34.60159626500321|
| 339.9007831078834| 34.73830208759114|
| 339.8350724565858| 34.65489429025321|
| 339.9006981638171|34.741711297466075|
|339.85114223901024| 34.60391043664569|
| 339.9176012834339|34.733835899266445|
|339.85677629540703|34.596872654451175|
|340.03999850033125|  34.7732916046863|
|339.86516714065425|34.606816308674404|
| 339.9019848293452|34.727662874436874|
+------------------+------------------+

root
 |-- coord_ra: double (nullable = true)
 |-- coord_decl: double (nullable = true)



In [12]:
val new_df = coordsDF.withColumn("htmid",HTMUtils.htmidUDF(old_df("coord_ra"),old_df("coord_decl"))).withColumn("healpixid", HEALPixUtils.healpixidUDF(old_df("coord_ra"), old_df("coord_decl")))

new_df.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

new_df: org.apache.spark.sql.DataFrame = [coord_ra: double, coord_decl: double ... 2 more fields]
+------------------+------------------+--------------+-------------+
|          coord_ra|        coord_decl|         htmid|    healpixid|
+------------------+------------------+--------------+-------------+
|347.75390124573187|34.006256639533504|13326611225421|3646029462090|
+------------------+------------------+--------------+-------------+
only showing top 1 row



In [11]:
val new_df = coordsDF.withColumn("htmid",HEALPixUtils.healpixidUDF(old_df("coord_ra"),old_df("coord_decl")))
new_df.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<console>:27: error: not found: value HEALPixUtils
       val new_df = coordsDF.withColumn("htmid",HEALPixUtils.healpixidUDF(old_df("coord_ra"),old_df("coord_decl")))
                                                ^

